In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn import metrics
from IPython.display import SVG
from graphviz import Source
from IPython.display import display

In [2]:
df = pd.read_csv('clean_data_final.csv')
df.head()

,type,E_Extroversion,S_Sensing,T_Thinking,J_Judging,posts,clean_posts,compound_sentiment,pos_sentiment,neg_sentiment,...,em,colons,emojis,word_count,unique_words,post_length_var,upper,link_count,ellipses,img_count
0,INFJ,0,0,0,1,'When asked of the things you wish you did ear...,asked thing wish earlier find answering...,0.999800,0.418667,0.136150,...,0.22,0.32,0.08,30.98,14.92,78.414931,1.46,0.04,0.62,0.00
1,INFJ,0,0,0,1,'I love both and they are equally important to...,love equally important music window soul in...,0.999950,0.600000,0.134585,...,0.36,0.14,0.00,28.58,12.72,160.744400,1.62,0.02,0.04,0.00
2,INFJ,0,0,0,1,Really? You think implying that everyone who i...,really think implying everyone entrepreneur s...,0.995499,0.353333,0.297340,...,0.10,0.50,0.20,22.28,11.76,176.249600,1.30,0.06,0.04,0.04
3,ENFJ,1,0,0,1,'Love is a crazy thing. Se is our best form ...,love crazy thing best form communication ...,0.999700,0.437333,0.262911,...,0.00,0.00,0.00,34.36,15.38,83.520000,2.10,0.02,0.18,0.00
4,INTP,0,0,1,0,'I am a physics undergrad with a computation e...,physic undergrad computation emphasis learni...,0.999700,0.424000,0.172144,...,0.48,0.14,0.00,23.90,12.66,152.750400,1.70,0.50,0.00,0.04


In [3]:
df.shape

(8344, 126)

In [4]:
df.isnull().sum()

type               0
E_Extroversion     0
S_Sensing          0
T_Thinking         0
J_Judging          0
                  ..
post_length_var    3
upper              3
link_count         3
ellipses           3
img_count          3
Length: 126, dtype: int64

In [5]:
df = df.dropna(how="any", axis=0)

In [6]:
df.isnull().sum()

type               0
E_Extroversion     0
S_Sensing          0
T_Thinking         0
J_Judging          0
                  ..
post_length_var    0
upper              0
link_count         0
ellipses           0
img_count          0
Length: 126, dtype: int64

In [7]:
df.dtypes

type                object
E_Extroversion       int64
S_Sensing            int64
T_Thinking           int64
J_Judging            int64
                    ...   
post_length_var    float64
upper              float64
link_count         float64
ellipses           float64
img_count          float64
Length: 126, dtype: object

In [8]:
#เซ็ต x y
X = df[
    [
        "compound_sentiment",
        "ADJ_avg",
        "ADP_avg",
        "ADV_avg",
        "CONJ_avg",
        "DET_avg",
        "NOUN_avg",
        "NUM_avg",
        "PRT_avg",
        "PRON_avg",
        "VERB_avg",
        "qm",
        "em",
        "colons",
        "emojis",
        "word_count",
        "unique_words",
        "upper",
        "link_count",
        "ellipses",
        "img_count",
    ]
]

Y = df.iloc[:, 0]

print(X.shape)
print(Y.shape)

(8341, 21)
(8341,)


In [9]:
X.head()

,compound_sentiment,ADJ_avg,ADP_avg,ADV_avg,CONJ_avg,DET_avg,NOUN_avg,NUM_avg,PRT_avg,PRON_avg,...,qm,em,colons,emojis,word_count,unique_words,upper,link_count,ellipses,img_count
0,0.999800,3.0,1.0,4.0,5.0,3.0,6.0,0.0,0.0,4.0,...,0.08,0.22,0.32,0.08,30.98,14.92,1.46,0.04,0.62,0.00
1,0.999950,2.0,1.0,3.0,5.0,2.0,5.0,0.0,0.0,5.0,...,0.12,0.36,0.14,0.00,28.58,12.72,1.62,0.02,0.04,0.00
2,0.995499,1.0,1.0,2.0,3.0,2.0,5.0,0.0,0.0,4.0,...,0.32,0.10,0.50,0.20,22.28,11.76,1.30,0.06,0.04,0.04
3,0.999700,2.5,0.0,3.0,4.0,2.5,6.0,0.0,0.0,5.0,...,0.04,0.00,0.00,0.00,34.36,15.38,2.10,0.02,0.18,0.00
4,0.999700,2.0,0.0,2.0,4.0,2.0,6.0,0.0,0.0,3.0,...,0.64,0.48,0.14,0.00,23.90,12.66,1.70,0.50,0.00,0.04


In [10]:
Y.head()

0    INFJ
1    INFJ
2    INFJ
3    ENFJ
4    INTP
Name: type, dtype: object

In [11]:
# Split dataset into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1) # 70% training and 30% test

# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy", max_depth=None)

# Train Decision Tree Classifer
clf = clf.fit(X_train,Y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
y_score = clf.score(X,Y)

In [13]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
ac = metrics.accuracy_score(Y_test, y_pred)
print('DecisionTree Accuracy: ', ac)

DecisionTree Accuracy:  0.1446264482620855
